In [2]:
# Importer les bibliothèques nécessaires
import osmnx as ox  # Bibliothèque pour récupérer et visualiser les réseaux de rues à partir d'OpenStreetMap
import networkx as nx  # Bibliothèque pour créer et gérer des réseaux/graphes complexes
from shapely.geometry import Point, LineString, MultiLineString  # Pour les opérations géométriques
from scipy.spatial import distance  # Pour les calculs de distance spatiale
from geopy.distance import geodesic  # Pour les calculs de distance géodésique (à la surface de la Terre)
from neo4j import GraphDatabase  # Pilote de base de données Neo4j
import numpy as np  # Pour les opérations numériques


# Preparation of graphs for storage in Neo4j

In [3]:
import numpy as np
from shapely.geometry import Point, LineString
from geopy.distance import geodesic

def calculate_distance(point1, point2):
    # Euclidean distance between two points
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def calculate_distance_in_meters(point1, point2):
    # Distance between points in meters (using a placeholder conversion)
    return calculate_distance(point1, point2) * 1000

def add_pharmacy_node(G, osmid, pharmacy_point):
    # Add a pharmacy node to the graph
    print('osmid : ', osmid)
    G.add_node(osmid, x=pharmacy_point.x, y=pharmacy_point.y, osmid=osmid)
    G.nodes[osmid]['pharmacy'] = 'yes'

def calculate_linestring_distance_in_meters(coords):
    # Total distance along a LineString in meters
    total_distance = 0.0
    for i in range(len(coords) - 1):
        distance = geodesic(coords[i], coords[i + 1]).meters
        total_distance += distance
    return total_distance

def calculate_distance(coord1, coord2):
    # Geodesic distance between two coordinates in meters
    return geodesic(coord1, coord2).meters

def linestring_to_list(geometry):
    # Convert LineString to list of coordinates
    return list(geometry.coords) if isinstance(geometry, LineString) else None

def projection_point_on_line(x1, y1, x2, y2, x3, y3):
    # Project point C onto line AB
    ABx, ABy = x2 - x1, y2 - y1
    ACx, ACy = x3 - x1, y3 - y1
    t = (ACx * ABx + ACy * ABy) / (ABx**2 + ABy**2)
    return x1 + t * ABx, y1 + t * ABy

def find_nearest_point(pharmacy_node, edge):
    # Find closest point on edge to pharmacy node
    distance, nearest_point = float('inf'), None
    for point in list(edge.coords):
        current_distance = geodesic((pharmacy_node.y, pharmacy_node.x), (point[1], point[0])).meters
        if current_distance < distance:
            distance, nearest_point = current_distance, point
    return nearest_point


In [4]:
def process_pharmacy(G, osmid, pharmacy_point):
    # Find the nearest edge to the pharmacy location
    nearest_edge = ox.distance.nearest_edges(G, pharmacy_point.x, pharmacy_point.y)
    u, v = nearest_edge[:2]
    edge_data = G.get_edge_data(*nearest_edge)
    
    if 'geometry' in edge_data:
        # If edge has geometry, modify it to include the pharmacy point
        coords_list = []
        edge_geom = LineString(list(edge_data['geometry'].coords).copy())
        nearest_point = find_nearest_point(pharmacy_point, edge_geom)
        edge_coords = list(edge_geom.coords)
        
        for coord in edge_coords:
            coords_list.append(coord)
            if calculate_distance(nearest_point, coord) < 0.1:
                coords_list.append((pharmacy_point.x, pharmacy_point.y))
                break
        
        edge_geom = LineString(coords_list)
        
    else:
        # If edge has no geometry, create a line between points
        edge_geom = LineString([
            (G.nodes[u]['x'], G.nodes[u]['y']),
            projection_point_on_line(
                G.nodes[u]['x'], G.nodes[u]['y'], 
                G.nodes[v]['x'], G.nodes[v]['y'], 
                pharmacy_point.x, pharmacy_point.y
            ),
            (pharmacy_point.x, pharmacy_point.y)
        ])
    
    # Define new edge data
    new_edge_data = {
        'reversed': True,
        'length': calculate_linestring_distance_in_meters(edge_geom.coords),
        'geometry': edge_geom
    }
    
    # Add the pharmacy node to the graph
    add_pharmacy_node(G, osmid, pharmacy_point)
    G.add_edge(u, osmid, **new_edge_data)
    
    # If the reverse edge exists and is marked as reversed, process it as well
    if v in G and u in G[v] and G[v][u] and edge_data['reversed']:
        edge_data = G[v][u][0]
        
        if 'geometry' in edge_data:
            # Adjust edge geometry if it has geometry data
            coords_list = []
            edge_geom = LineString(list(edge_data['geometry'].coords).copy())
            nearest_point = find_nearest_point(pharmacy_point, edge_geom)
            edge_coords = list(edge_geom.coords)
            
            for coord in edge_coords:
                coords_list.append(coord)
                if calculate_distance(nearest_point, coord) < 0.1:
                    coords_list.append((pharmacy_point.x, pharmacy_point.y))
                    break
            
            edge_geom = LineString(coords_list)
        
        else:
            # If no geometry, create a simple line with the pharmacy point
            edge_geom = LineString([
                (G.nodes[u]['x'], G.nodes[u]['y']),
                projection_point_on_line(
                    G.nodes[u]['x'], G.nodes[u]['y'], 
                    G.nodes[v]['x'], G.nodes[v]['y'], 
                    pharmacy_point.x, pharmacy_point.y
                ),
                (pharmacy_point.x, pharmacy_point.y)
            ])
    
    # Add the modified reverse edge to the graph
    new_edge_data = {
        'reversed': True,
        'length': calculate_linestring_distance_in_meters(edge_geom.coords),
        'geometry': edge_geom
    }
    G.add_edge(v, osmid, **new_edge_data)


In [5]:
G = ox.graph_from_place('Fès, Fès-Meknès, Maroc', network_type='drive')

In [6]:
2# Identifier les pharmacies en utilisant la nouvelle fonction
pharmacies = ox.geometries_from_place('Fès, Fès-Meknès, Maroc', tags={'amenity': 'pharmacy'})

C:\Users\oussama ab\AppData\Local\Temp\ipykernel_15740\2857674731.py:2: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  pharmacies = ox.geometries_from_place('Fès, Fès-Meknès, Maroc', tags={'amenity': 'pharmacy'})


In [7]:
pharmacy_points = [
    ( index[1],row['geometry'])
    for index, row in pharmacies.iterrows()
]


In [9]:
a = 1
for i,j in pharmacy_points:
    print(a,end='->')
    process_pharmacy(G,i,j)
    a+=1

1->osmid :  1866502248
2->osmid :  2328748922
3->osmid :  3718016810
4->osmid :  4606353707
5->osmid :  4681620692
6->osmid :  4698968089
7->osmid :  4711448391
8->osmid :  4711868990
9->osmid :  4711934790
10->osmid :  4711934894
11->osmid :  4712385991
12->osmid :  4712445890
13->osmid :  4712508653
14->osmid :  4724813489
15->osmid :  4725748199
16->osmid :  4729539594
17->osmid :  4730950544
18->osmid :  4738031421
19->osmid :  4738681626
20->osmid :  4740427422
21->osmid :  4741642523
22->osmid :  4744887621
23->osmid :  4746773124
24->osmid :  4746889021
25->osmid :  4753542728
26->osmid :  4757773462
27->osmid :  4759451623
28->osmid :  4759495424
29->osmid :  4763995654
30->osmid :  4772565121
31->osmid :  4774537683
32->osmid :  4788557121
33->osmid :  4788557323
34->osmid :  4797223421
35->osmid :  4801102022
36->osmid :  4801209322
37->osmid :  4802070122
38->osmid :  4803244621
39->osmid :  4803711989
40->osmid :  4805354921
41->osmid :  4805418523
42->osmid :  4805680322
4

333->osmid :  9062947497
334->osmid :  9062954266
335->osmid :  9062971624
336->osmid :  9063042255
337->osmid :  9063046705
338->osmid :  9063050284
339->osmid :  9064150951
340->osmid :  9064151371
341->osmid :  9064248041
342->osmid :  9064274028
343->osmid :  9067076233
344->osmid :  9067097844
345->osmid :  9067122685
346->osmid :  9067461578
347->osmid :  9074295920
348->osmid :  9077198847
349->osmid :  9077218586
350->osmid :  9077240390
351->osmid :  9077291355
352->osmid :  9077313479
353->osmid :  9077395032
354->osmid :  9080092417
355->osmid :  9080252241
356->osmid :  9080367514
357->osmid :  9080513814
358->osmid :  9082210309
359->osmid :  9082226035
360->osmid :  9082234019
361->osmid :  9082250910
362->osmid :  9082252419
363->osmid :  9082305309
364->osmid :  9088108976
365->osmid :  9088262543
366->osmid :  9088494169
367->osmid :  9091504397
368->osmid :  9091558885
369->osmid :  9091566947
370->osmid :  9091590534
371->osmid :  9091645325
372->osmid :  9091663749


In [10]:
import numpy as np
from shapely.geometry import LineString, Point

def project_and_insert_point(path_coords, point_coords):
    """
    Project a point onto a path and insert it at the correct position in the path's coordinates.
    
    :param path_coords: List of tuples representing path coordinates [(x1, y1), (x2, y2), ...]
    :param point_coords: Tuple representing the point's coordinates (x, y)
    :return: New list of path coordinates with the projected point inserted
    """
    path = LineString(path_coords)
    point = Point(point_coords)
    
    # Find the projected position of the point on the path
    distance = path.project(point)
    projected_point = path.interpolate(distance)
    projected_coords = (projected_point.x, projected_point.y)
    
    # Insert the projected point in the correct segment of the path
    new_path_coords = []
    inserted = False
    for i in range(len(path_coords) - 1):
        segment_start = Point(path_coords[i])
        segment_end = Point(path_coords[i + 1])
        segment = LineString([segment_start, segment_end])
        
        new_path_coords.append(path_coords[i])
        
        # Check if the projected point should be inserted in this segment
        if not inserted and segment.project(projected_point) < segment.length:
            new_path_coords.append(projected_coords)
            inserted = True
    
    new_path_coords.append(path_coords[-1])  # Add the last point of the path
    
    # If the point wasn't inserted, add it to the end
    if not inserted:
        new_path_coords.append(projected_coords)

    print(projected_coords)
    return new_path_coords

# Example usage
path = [(0, 0), (1, 2), (2, 1), (3, 3)]  # Non-linear path
point = (2, 2)

new_path = project_and_insert_point(path, point)
print("The new path with the projected point is:")
for coord in new_path:
    print(coord)


(2.4, 1.7999999999999998)
The new path with the projected point is:
(0, 0)
(1, 2)
(2.4, 1.7999999999999998)
(2, 1)
(3, 3)


# Store the graph in Neo4j.

In [17]:
from neo4j import GraphDatabase

# Function to convert OSMnx graph nodes to Neo4j data
def convert_nodes_osmnx_graph_to_neo4j(G):
    # Connect to Neo4j database
    uri = "bolt://localhost:7687"
    user = "oussa"
    password = "oussa1234"
    driver = GraphDatabase.driver(uri, auth=(user, password))
    
    # Create a Neo4j session
    with driver.session(database="pharma") as session:
        # Loop through each node in the OSMnx graph
        for node in G.nodes(data=True):
            # Prepare node properties
            properties = {
                'osmid': node[0],
                'y': node[1].get('y'),
                'x': node[1].get('x')
            }
            
            # Check if 'pharmacy' attribute exists
            if 'pharmacy' in node[1]:
                properties['pharmacy'] = node[1]['pharmacy']

            # Build Cypher query to create the node
            query = "CREATE (a:Location {osmid: $osmid, y: $y, x: $x"
            if 'pharmacy' in properties:
                query += ", pharmacy: $pharmacy"
            query += "})"
            
            # Filter valid properties (exclude callable and None values)
            valid_properties = {k: v for k, v in properties.items() if not callable(v) and v is not None}
            
            # Execute the query with node properties
            session.run(query, **valid_properties)
    
    # Close the connection
    driver.close()

# Call the function to convert the graph
convert_nodes_osmnx_graph_to_neo4j(G)


C:\Users\oussama ab\AppData\Roaming\Python\Python311\site-packages\neo4j\_sync\driver.py:544: ResourceWarning: unclosed  BoltDriver: <neo4j._sync.driver.BoltDriver object at 0x00000209F8B61490>.
  _unclosed_resource_warn(self)
C:\Users\oussama ab\AppData\Roaming\Python\Python311\site-packages\neo4j\_sync\driver.py:549: DeprecationWarning: Relying on Driver's destructor to close the session is deprecated. Please make sure to close the session. Use it as a context (`with` statement) or make sure to call `.close()` explicitly. Future versions of the driver will not close drivers automatically.
  _deprecation_warn(
C:\Users\oussama ab\AppData\Roaming\Python\Python311\site-packages\neo4j\_sync\driver.py:544: ResourceWarning: unclosed  BoltDriver: <neo4j._sync.driver.BoltDriver object at 0x00000209ED3C8910>.
  _unclosed_resource_warn(self)
C:\Users\oussama ab\AppData\Roaming\Python\Python311\site-packages\neo4j\_sync\driver.py:544: ResourceWarning: unclosed  BoltDriver: <neo4j._sync.driver.B

In [19]:
# Function to convert OSMnx graph edges to Neo4j data
def convert_edges_osmnx_graph_to_neo4j(G):
    # Connect to Neo4j database
    uri = "bolt://localhost:7687"
    user = "oussa"
    password = "oussa1234"
    driver = GraphDatabase.driver(uri, auth=(user, password))
    
    # Create a Neo4j session
    with driver.session(database="pharma") as session:
        for u, v, k, data in G.edges(keys=True, data=True):
            # Prepare edge properties
            edge_properties = {
                'length': data.get('length', 0)
            }
            
            # Check if geometry is present and format it
            if 'geometry' in data:
                edge_properties['geometry'] = ""
                coords = list(data['geometry'].coords)
                for coord in coords:
                    edge_properties['geometry'] += f"|{coord[0]}#{coord[1]}"
            else:
                # Default geometry if none is present
                edge_properties['geometry'] = f"|{G._node[u]['x']}#{G._node[u]['y']}|{G._node[v]['x']}#{G._node[v]['y']}"
            
            # Add additional attributes if available
            if 'name' in data:
                edge_properties['name'] = data['name']
            if 'highway' in data:
                edge_properties['highway'] = data['highway']

            # Create relationship in Neo4j for each OSMnx edge
            query = """
            MATCH (a:Location {osmid: $u}), (b:Location {osmid: $v})
            CREATE (a)-[:ROAD {length: $length"""
            if 'geometry' in edge_properties:
                query += ", geometry: $geometry"
            if 'name' in edge_properties:
                query += ", name: $name"
            if 'highway' in edge_properties:
                query += ", highway: $highway"
            query += "}]->(b)"

            # Execute the query
            session.run(query, u=u, v=v, **edge_properties)

    # Close the connection
    driver.close()

# Call the function to convert the edges
convert_edges_osmnx_graph_to_neo4j(G)
